In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import dash
import plotly.express as px
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import warnings
import dash_table

In [2]:
Stats = pd.read_csv("Stats.csv", sep=";")
Summary = pd.read_csv("Summary.csv", sep=";")
Teams = pd.read_csv("Teams.csv", sep= ";")
Players = pd.read_csv("PlayerList.csv", sep= ";")

In [7]:
def mergeStats(Stats, Players):
    #TIME
    Scored = Stats[(Stats["Co"] == "Gol") & (Stats["Info"] != "samobój")].groupby(by=["Kto"])["Co"].count().reset_index()
    ScoredPenalty = Stats[(Stats["Co"] == "Gol") & (Stats["Info"] == "rzut karny")].groupby(by=["Kto"])["Co"].count().reset_index()
    MissedPenalty = Stats[(Stats["Co"] == "Niestrzelony karny")].groupby(by=["Kto"])["Co"].count().reset_index()
    OwnGoal = Stats[(Stats["Co"] == "Gol") & (Stats["Info"] == "samobój")].groupby(by=["Kto"])["Co"].count().reset_index()
    Yellow = Stats[(Stats["Co"] == "Zolta kartka")].groupby(by=["Kto"])["Co"].count().reset_index()
    Red = Stats[(Stats["Co"] == "Czerwona")].groupby(by=["Kto"])["Co"].count().reset_index()

    #rename columns
    Scored.columns = ['id', "Bramka"] 
    ScoredPenalty.columns = ['id', "Karny"]
    MissedPenalty.columns = ["id", "Pudlo"]
    OwnGoal.columns = ['id','Samoboj']
    Yellow.columns = ['id', "Zolta"]
    Red.columns = ['id', "Czerwona"]

    tempPlayers = Players
    #MERGE
    PlayersStats = pd.merge(tempPlayers,Scored, how="left", on="id" )
    PlayersStats = pd.merge(PlayersStats,ScoredPenalty,how="left", on='id' )
    PlayersStats = pd.merge(PlayersStats,MissedPenalty,how="left", on='id' )
    PlayersStats = pd.merge(PlayersStats,OwnGoal,how="left", on='id' )
    PlayersStats = pd.merge(PlayersStats,Yellow,how="left", on='id' )
    PlayersStats = pd.merge(PlayersStats,Red,how="left", on='id' )
    return PlayersStats

In [48]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("Gdzie"),
                dcc.Dropdown(id="Where", 
                options=[{'label': "Dom", "value":"Dom"},
                         {'label': "Wyjazd", "value":"Wyjazd"},
                         {'label': "Oba", "value":"Oba"}
                        ],
                value= "Dom"),
                
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Bramki"),
                dcc.Dropdown(id="Score", 
                options=[{'label': "Stracone", "value":"Stracone"},
                         {'label': "Zdobyte", "value":"Zdobyte"},
                         {'label': "Bilans", "value":"Oba"}
                        ],
                value= "Zdobyte"),
                
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Kolejki"),
                dcc.RangeSlider(
                    id='Kolejki',
                    min=1,
                    max=30,
                    step=1,
                    value=[1, 30]
                ),
                html.Div(id='output-container-range-slider'),   
            ]
        ),
    ],
    body = True,
)
app.layout = dbc.Container(
    [
        html.H1("Liczba zdobytych bramek w sezonie"),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls, md=4),
                dbc.Col(dcc.Graph(id='histogram', figure = px.histogram(data_frame=Stats, x = "Zespol1-Nazwa",
                                                   y = "Zespol1-Bramki")), md=8),
            ],
            align="center",
        ),
    ],
    fluid=True,
)

@app.callback(
    Output('output-container-range-slider', 'children'),
    Output(component_id="histogram", component_property="figure"),
    [
        Input('Kolejki', 'value'),
        Input(component_id="Where", component_property="value"),
        Input(component_id="Score", component_property="value"),
    ],
)
def CreateDashboard(Kolejki, Where, Score):
    who1 = ''
    score1 = ''
    #add description
    description = "Kolejki od " + str(Kolejki[0]) + " do " + str(Kolejki[1])
    #narrows rounds
    StatsForDash = Stats[(Stats["Kolejka"] >=Kolejki[0]) & (Stats["Kolejka"] <=Kolejki[1])]
    figure = px.histogram()
    #Wyjazd
    if Where == "Wyjazd":
        #Zdobyte
        if Score == "Zdobyte":
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol2-Nazwa", y = "Zespol2-Bramki").update_xaxes(
                categoryorder='total descending')
        #Stracone
        elif Score == "Stracone":
        #Bilans wyjazd
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol2-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
        else:
            Goals1 = StatsForDash[["Zespol2-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals1["Zespol1-Bramki"] = Goals1["Zespol2-Bramki"] - Goals2["Zespol1-Bramki"]
            figure = px.histogram(data_frame=Goals1, x = "Zespol2-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
    #Dom
    elif Where == "Dom":
        #Zdobyte
        if Score == "Zdobyte":
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
            #Stracone
        elif Score == "Stracone":
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol1-Nazwa", y = "Zespol2-Bramki").update_xaxes(
                categoryorder='total descending')
        #Bilans Dom
        else:
            Goals1 = StatsForDash[["Zespol1-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol1-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals1["Zespol1-Bramki"] = Goals1["Zespol1-Bramki"] - Goals2["Zespol2-Bramki"]
            figure = px.histogram(data_frame=Goals1, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
    #Dom + Wyjazd
    else:
        #Zdobyte
        if Score == "Zdobyte":
            Goals1 = StatsForDash[["Zespol1-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol1-Bramki"]
            Goals1 = Goals1.append(Goals2, ignore_index=True)
            Goals1 = Goals1.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            figure = px.histogram(data_frame=Goals1, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
        #Stracone
        elif Score == "Stracone":
            Goals1 = StatsForDash[["Zespol1-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol2-Bramki"]
            Goals1 = Goals1.append(Goals2, ignore_index=True)
            Goals1 = Goals1.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            figure = px.histogram(data_frame=Goals1, x = "Zespol1-Nazwa", y = "Zespol2-Bramki").update_xaxes(
                categoryorder='total descending')
        else:
            #Zdobyte
            GoalsZ = StatsForDash[["Zespol1-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol1-Bramki"]
            GoalsZ = GoalsZ.append(Goals2, ignore_index=True)
            GoalsZ = GoalsZ.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            
            #Stracone
            GoalsS = StatsForDash[["Zespol1-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol2-Bramki"]
            GoalsS = GoalsS.append(Goals2, ignore_index=True)
            GoalsS = GoalsS.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            
            GoalsZ["Zespol1-Bramki"] =  GoalsZ["Zespol1-Bramki"] - GoalsS["Zespol2-Bramki"]
            
            figure = px.histogram(data_frame=GoalsZ, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
    figure.update_layout(
        xaxis_title="Zespoly",
        yaxis_title="Liczba bramek",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    return description, figure

         
app.run_server(port=8050)

Dash app running on http://127.0.0.1:8050/


In [47]:
app2 = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("Team"),
                dcc.Dropdown(id="Team", 
                options=[{'label':x, "value":x}
                         for x in sorted(Stats["Zespol1-Nazwa"].unique())
                        ],
                value= "Legia Warszawa"),
                
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Kolejki"),
                dcc.RangeSlider(
                    id='Kolejki2',
                    min=1,
                    max=30,
                    step=1,
                    value=[1, 30]
                ),
                html.Div(id='output-container-range-slider2'),   
            ]
        ),
    ],
    body = True,
)
app2.layout = dbc.Container(
    [
        html.H1("Statystyki dotyczace klubow"),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls),
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col( html.H2("Skumulowana liczba bramek"),
                width={"size": 4, "offset": 1}
                       )
               
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id="GoalHome",figure = {}), md=6),
                dbc.Col(dcc.Graph(id="GoalAway",figure = {}),md=6),
            ],
            align="center",
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col( html.H2("Posiadanie piłki"),align = "center",
                width={"size": 4, "offset": 1}
                       )
               
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id="Possesion",figure = {})),
            ],
            align="center",
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col( html.H2("Strzały i strzały celne"),
                width={"size": 4, "offset": 1}
                       )
               
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id="ShotsHome",figure = {}), md=6),
                dbc.Col(dcc.Graph(id="ShotsAway",figure = {}),md=6),
            ],
            align="center",
        ),
        dbc.Row(
            [
                dbc.Col(html.H3(id="ShotsStatsHome"), width={"size": 6, "offset":1}),
                dbc.Col(html.H3(id="ShotsStatsAway"),width={"size": 4, "offset": 0}),
            ],
            align="center",
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col( html.H2("Faule i kartki"),align = "center",
                width={"size": 4, "offset": 1}
                       )
               
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id="FaulsHome",figure = {})),
                dbc.Col(dcc.Graph(id="FaulsAway",figure = {})),
            ],
            align="center",
        ),
    ],
    fluid=True,
)
@app2.callback(
    Output('output-container-range-slider2', 'children'),
    Output(component_id="GoalHome", component_property="figure"),
    Output(component_id="GoalAway", component_property="figure"),
    Output(component_id="Possesion", component_property="figure"),
    Output(component_id="ShotsHome", component_property="figure"),
    Output(component_id="ShotsAway", component_property="figure"),
    Output('ShotsStatsHome', 'children'),
    Output('ShotsStatsAway', 'children'),
    Output(component_id="FaulsHome", component_property="figure"),
    Output(component_id="FaulsAway", component_property="figure"),
    [
        Input('Kolejki2', 'value'),
        Input('Team', 'value'),
    ],
)
def CreateDashboard(Kolejki, Team):
    #add description
    description = "Kolejki od " + str(Kolejki[0]) + " do " + str(Kolejki[1])
    
    #narrows rounds
    StatsForDash = Stats[(Stats["Kolejka"] >=Kolejki[0]) & (Stats["Kolejka"] <=Kolejki[1])]
    
    clubScore = StatsForDash
    clubScore = clubScore.sort_values(by=['Kolejka'])
    
    #GOALS DATA VISUALIZATION WITH PLOTLY
    clubScoreHome = clubScore[(clubScore["Zespol1-Nazwa"] == Team)]
    clubScoreAway = clubScore[(clubScore["Zespol2-Nazwa"] == Team)]
    #Scored
    warnings.filterwarnings("ignore") 
    clubScoreHome['GoleZdobyteSkumulowane'] = [clubScoreHome["Zespol1-Bramki"][clubScoreHome["Kolejka"]<=i].sum() for i in clubScoreHome["Kolejka"]]
    clubScoreAway['GoleZdobyteSkumulowane'] = [clubScoreAway["Zespol2-Bramki"][clubScoreAway["Kolejka"]<=i].sum() for i in clubScoreAway["Kolejka"]]
    #Lost
    clubScoreHome['GoleStraconeSkumulowane'] = [clubScoreHome["Zespol2-Bramki"][clubScoreHome["Kolejka"]<=i].sum() for i in clubScoreHome["Kolejka"]]
    clubScoreAway['GoleStraconeSkumulowane'] = [clubScoreAway["Zespol1-Bramki"][clubScoreAway["Kolejka"]<=i].sum() for i in clubScoreAway["Kolejka"]]
    
    figHome = go.Figure()
    figHome.add_trace(go.Scatter(x=clubScoreHome["Kolejka"], y=clubScoreHome["GoleZdobyteSkumulowane"],
                        mode='lines+markers',
                        name='Zdobyte'))
    figHome.add_trace(go.Scatter(x=clubScoreHome["Kolejka"], y=clubScoreHome["GoleStraconeSkumulowane"],
                        mode='lines+markers',
                        name='Stracone'))
    figHome.update_layout(
        title="W meczach domowych",
        xaxis_title="Kolejki",
        yaxis_title="Liczba bramek",
        legend_title="Bramki",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    figAway = go.Figure()
    figAway.add_trace(go.Scatter(x=clubScoreAway["Kolejka"], y=clubScoreAway["GoleZdobyteSkumulowane"],
                        mode='lines+markers',
                        name='Zdobyte'))
    figAway.add_trace(go.Scatter(x=clubScoreAway["Kolejka"], y=clubScoreAway["GoleStraconeSkumulowane"],
                        mode='lines+markers',
                        name='Stracone'))
    figAway.update_layout(
        title="W meczach wyjazdowych",
        xaxis_title="Kolejki",
        yaxis_title="Liczba bramek",
        legend_title="Bramki",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    
    #POSSESION DATA VISUALIZATION WITH PLOTLY
    
    clubPossesion = StatsForDash
    clubPossesion = clubPossesion.sort_values(by=['Kolejka'])
    
    clubPossesionHome = clubPossesion[(clubPossesion["Zespol1-Nazwa"] == Team)]
    clubPossesionAway = clubPossesion[(clubPossesion["Zespol2-Nazwa"] == Team)]
    
    PossesionHome = clubPossesionHome["Zespol1-Posiadanie piłki"].agg('mean')
    PossesionAway = clubPossesionAway["Zespol2-Posiadanie piłki"].agg('mean')
    PossesionAll = (PossesionAway + PossesionHome)/2
    
    figPossesion = go.Figure()
    figPossesion.add_trace(go.Bar(
        y=['Wyjazd', 'Dom', "Ogolna"],
        x=[PossesionAway, PossesionHome, PossesionAll],
        name=Team,
        orientation='h',
        marker=dict(
            color='rgba(246, 78, 139, 0.6)',
            line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
        )
    ))
    figPossesion.add_trace(go.Bar(
        y=['Wyjazd', 'Dom', "Ogolna"],
        x=[100-PossesionAway, 100-PossesionHome, 100-PossesionAll],
        name='Rywale',
        orientation='h',
         marker=dict(
            color='rgba(58, 71, 80, 0.6)',
            line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
        )
    ))
    figPossesion.update_layout(barmode='stack',
                               font=dict(
                                   family="Courier New, monospace",
                                   size=18,
                                   color="RebeccaPurple"
            ))
    
    #SHOTS DATA VISUALIZATION WITH PLOTLY
    clubShotsHome = clubScore[(clubScore["Zespol1-Nazwa"] == Team)]
    clubShotsAway = clubScore[(clubScore["Zespol2-Nazwa"] == Team)]
    figHomeShots = go.Figure()
    figHomeShots.add_trace(go.Scatter(x=clubShotsHome["Kolejka"], y=clubShotsHome["Zespol1-Strzały"],
                        mode='lines+markers',
                        name='W sumie'))
    figHomeShots.add_trace(go.Scatter(x=clubShotsHome["Kolejka"], y=clubShotsHome["Zespol1-Strzały celne"],
                        mode='lines+markers',
                        name='Celne'))
    figHomeShots.update_layout(
        title="W meczach domowych",
        xaxis_title="Kolejki",
        yaxis_title="Liczba strzałów",
        legend_title="Strzały",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    figAwayShots = go.Figure()
    figAwayShots.add_trace(go.Scatter(x=clubShotsAway["Kolejka"], y=clubShotsAway["Zespol2-Strzały"],
                        mode='lines+markers',
                        name='W sumie'))
    figAwayShots.add_trace(go.Scatter(x=clubShotsAway["Kolejka"], y=clubShotsAway["Zespol2-Strzały celne"],
                        mode='lines+markers',
                        name='Celne'))
    figAwayShots.update_layout(
        title="W meczach wyjazdowych",
        xaxis_title="Kolejki",
        yaxis_title="Liczba strzałów",
        legend_title="Strzały",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    
    #SHOTS ACCURACY
    homeS = clubShotsHome['Zespol1-Strzały'].sum()
    awayS = clubShotsAway['Zespol2-Strzały'].sum()
    shotsHome = 0
    shotsAway = 0
    
    if homeS != 0:
        shotsHome = clubShotsHome['Zespol1-Strzały celne'].sum() / homeS
    if awayS != 0:
        shotsAway = clubShotsAway['Zespol2-Strzały celne'].sum() / awayS
    
    shotsHome = str(round(shotsHome, 2)) + "% celności strzałów"
    shotsAway = str(round(shotsAway, 2)) + "% celności strzałów"
    
    
    #SHOTS DATA VISUALIZATION WITH PLOTLY
    clubFaulsHome = clubScore[(clubScore["Zespol1-Nazwa"] == Team)]
    clubFaulsAway = clubScore[(clubScore["Zespol2-Nazwa"] == Team)]
    figFaulHome = go.Figure()
    figFaulHome.add_trace(go.Scatter(x=clubFaulsHome["Kolejka"], y=clubFaulsHome["Zespol2-Faule"],
                        mode='lines+markers',
                        name='Faule'))
    figFaulHome.add_trace(go.Scatter(x=clubFaulsHome["Kolejka"], y=clubFaulsHome["Zespol2-Kartki"],
                        mode='lines+markers',
                        name='Kartki'))
    figFaulHome.add_trace(go.Scatter(x=clubFaulsHome["Kolejka"], y=clubFaulsHome["Zespol1-Rzuty wolne"],
                        mode='lines+markers',
                        name='Rzuty wolne'))
    figFaulHome.update_layout(
        title="W meczach domowych",
        xaxis_title="Kolejki",
        yaxis_title="Faule/Kartki",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    figFaulAway = go.Figure()
    figFaulAway.add_trace(go.Scatter(x=clubFaulsAway["Kolejka"], y=clubFaulsAway["Zespol1-Faule"],
                        mode='lines+markers',
                        name='Faule'))
    figFaulAway.add_trace(go.Scatter(x=clubFaulsAway["Kolejka"], y=clubFaulsAway["Zespol1-Kartki"],
                        mode='lines+markers',
                        name='Kartki'))
    figFaulAway.add_trace(go.Scatter(x=clubFaulsAway["Kolejka"], y=clubFaulsAway["Zespol2-Rzuty wolne"],
                        mode='lines+markers',
                        name='Rzuty wolne'))
    figFaulAway.update_layout(
        title="W meczach wyjazdowych",
        xaxis_title="Kolejki",
        yaxis_title="Faule/Kartki",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="RebeccaPurple"
        )
    )
    return description, figHome, figAway, figPossesion, figHomeShots, figAwayShots, shotsHome, shotsAway, figFaulHome, figFaulAway

app2.run_server(port=8051)

Dash app running on http://127.0.0.1:8051/


In [22]:
warnings.filterwarnings("ignore")
now = pd.Timestamp('now')
PlayersAge = Players[Players["Data urodzenia"].isnull() == False]
PlayersAge['Data urodzenia'] = pd.to_datetime(PlayersAge['Data urodzenia'], format='%Y-%m-%d')
PlayersAge['Data urodzenia'] = PlayersAge['Data urodzenia'].where(PlayersAge['Data urodzenia'] < now, PlayersAge['Data urodzenia'] -  np.timedelta64(100, 'Y'))
PlayersAge['Wiek'] = (now - PlayersAge['Data urodzenia']).astype('<m8[Y]')
PlayersAge = mergeStats(Summary, PlayersAge)
newLabels = np.append(Stats["Zespol1-Nazwa"].unique(), "Wszyscy")
minHeight = Players[Players['Wzrost'] != 0]["Wzrost"].min()
maxHeight = Players[Players['Wzrost'] != 0]["Wzrost"].max()

In [39]:
tabColumns = ['imie', 'nazwisko', 'Obecny klub','Pozycja', 'Wzrost', 'Noga', 'Waga', 'Wiek', 'Bramka', 'Karny', 'Pudlo','Samoboj', 'Zolta', 'Czerwona']
PlayersForTab = PlayersAge[tabColumns].fillna(0)

In [46]:
warnings.filterwarnings("ignore")
app3 = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

tab = dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i}
            for i in PlayersForTab.columns
        ],
        data = PlayersForTab.to_dict("records"),
#         editable=True,              # allow editing of data inside all cells
#         filter_action="native",     # allow filtering of data by user ('native') or not ('none')
         sort_action="native",       # enables data to be sorted per-column by user or not ('none')
        sort_mode="single",         # sort across 'multi' or 'single' columns
#         column_selectable="multi",  # allow users to select 'multi' or 'single' columns
#         row_selectable="multi",     # allow users to select 'multi' or 'single' rows
#         row_deletable=True,         # choose if user can delete a row (True) or not (False)
#         selected_columns=[],        # ids of columns that user selects
#         selected_rows=[],           # indices of rows that user selects
        page_action="native",       # all data is passed to the table up-front or not ('none')
        page_current=0,             # page number that user is on
        page_size=11,               # number of rows visible per page
        style_cell={                # ensure adequate header width when text is shorter than cell's text
            'minWidth': 95, 'maxWidth': 95, 'width': 95
        },
        style_data={                # overflow cells' content into multiple lines
            'whiteSpace': 'normal',
            'height': 'auto'
        }
    ),
controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("Statystyka"),
                dcc.Dropdown(id="Stat", 
                options=[{'label': "Bramka", "value":"Bramka"},
                         {'label': "Karny", "value":"Karny"},
                         {'label': "Spudłowany karny", "value":"Pudlo"},
                         {'label': "Samobój", "value":"Samoboj"},
                         {'label': "Żółta kartka ", "value":"Zolta"},
                         {'label': "Czerwona kartka ", "value":"Czerwona"}
                        ],
                value= "Bramka"),  
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Zespół"),
                dcc.Dropdown(id="Team2", 
                options=[
                         {'label':x, "value":x}
                         for x in sorted(newLabels)
                        ],
                value= "Wszyscy", multi = True),
                
            ]
        ),
         dbc.FormGroup(
            [
                dbc.Label("Minuty"),
                dcc.RangeSlider(
                    id='Time',
                    min=0,
                    max=90,
                    step=1,
                    value=[0, 90]
                ),
                html.Div(id='TimeLabel'),   
            ]
        ),
         dbc.FormGroup(
            [
                dbc.Label("Wzrost"),
                dcc.RangeSlider(
                    id='Height',
                    min=minHeight,
                    max=maxHeight,
                    step=0.5,
                    value=[minHeight, maxHeight]
                ),
                html.Div(id='HeightLabel'),   
            ]
        ),
    ],
    body = True,
)
controls2 = dbc.Card(
    [ 
        dbc.FormGroup(
            [
                dbc.Label("Zespół"),
                dcc.Dropdown(id="Team3", 
                options=[
                         {'label':x, "value":x}
                         for x in sorted(newLabels)
                        ],
                value= "Wszyscy", multi = True),
                
            ]
        ),
         dbc.FormGroup(
            [
                dbc.Label("Wzrost"),
                dcc.RangeSlider(
                    id='Height2',
                    min=minHeight,
                    max=maxHeight,
                    step=0.5,
                    value=[minHeight, maxHeight]
                ),
                html.Div(id='HeightLabel2'),   
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Wiek"),
                dcc.RangeSlider(
                    id='Age',
                    min=15,
                    max=45,
                    step=1,
                    value=[15, 22]
                ),
                html.Div(id='AgeLabel'),   
            ]
        ),
    ],
    body = True,
)
controls3 = dbc.Card(
    [ 
        dbc.FormGroup(
            [
                dbc.Label("Zespół"),
                dcc.Dropdown(id="Team4", 
                options=[
                         {'label':x, "value":x}
                         for x in sorted(newLabels)
                        ],
                value= "Wszyscy", multi = True),
                
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Wiek"),
                dcc.RangeSlider(
                    id='Age2',
                    min=15,
                    max=45,
                    step=1,
                    value=[15, 22]
                ),
                html.Div(id='AgeLabel2'),   
            ]
        ),
    ],
    body = True,
)
app3.layout = dbc.Container(
    [
        html.H1("Statystyki z podziałem na zawodników "),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col( html.H2("Wszystkie statystyki"),align = "center",
                width={"size": 4, "offset": 1}
                       )
               
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls, md=4),
                dbc.Col(dcc.Graph(id="StatsPlot",figure = {})),
            ],
            align="center",
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col( html.H2("Statystyki bramkowe"),align = "center",
                width={"size": 4, "offset": 1}
                       )
                
               
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls2, md=4),
                dbc.Col(dcc.Graph(id="StatsPlot2",figure = {})),
            ],
            align="center",
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col( html.H2("Podsumowanie statystyk"),align = "center",
                width={"size": 4, "offset": 1}
                       )
                
               
            ]
        ),
        html.Hr(),
         html.Div(
                [
                dbc.Row(
                [
                    dbc.Col(
                        [
                            dbc.FormGroup(
                            [
                                dbc.Label("Minuty"),
                                dcc.RangeSlider(
                                    id='Time2',
                                    min=0,
                                    max=90,
                                    step=1,
                                    value=[0, 90]
                                ),
                                html.Div(id='TimeLabel2'),   
                            ]
        ),
                        ]
                    )
                ]),
                dbc.Row(
                    [
                        dbc.Col(html.Div(dbc.Alert("Bramki: ",id="Alert1", color="primary"),), width=4),
                        dbc.Col(html.Div(dbc.Alert("Karne strzelone: ",id="Alert2", color="success"),),width=4),
                        dbc.Col(html.Div(dbc.Alert("Karne niestrzelone: ",id="Alert3", color="secondary"),),width=4),
                    ],
                    align="start",
                ),
                dbc.Row(
                    [
                        dbc.Col(html.Div(dbc.Alert("Żółte kartki: ",id="Alert4", color="warning"),),width=4),
                        dbc.Col(html.Div(dbc.Alert("Czerwone kartki: ",id="Alert5", color="danger"),),width=4),
                        dbc.Col(html.Div(dbc.Alert("Samobóje: ",id="Alert6", color="dark"),),width=4),
                    ],
                    align="center",
                ),
                ], style={'textAlign': 'center', 'font-size':25}
                ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col( html.H2("Najlepsi zawodnicy"),align = "center",
                width={"size": 4, "offset": 1}
                       )
                
               
            ]
        ),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls3, md=2),
                dbc.Col(tab),
            ],
        ),
        html.Hr(),
        html.Br(),
        html.Br(),
        html.Br(),
        
    ],
    fluid=True,
)
@app3.callback(
    Output(component_id="StatsPlot", component_property="figure"),
    Output(component_id="TimeLabel", component_property="children"),
    Output(component_id="HeightLabel", component_property="children"),
    [
        Input('Stat', 'value'),
        Input("Team2",'value'),
        Input("Time", "value"),
        Input("Height", "value"),  
    ],
)
def CreateDashboard(Stat, Team, Time, Height):
    #add description
    descriptionTime = "Minuty od " + str(Time[0]) + " do " + str(Time[1])
    descriptionHeight = "Wzrost od " + str(Height[0]) + " do " + str(Height[1])
    
    test = Summary[(Summary["Czas"] >=Time[0]) &  (Summary["Czas"] <= Time[1])]
    PlayersStats = mergeStats(test, Players)
    #FisrtPlot
    PickdTeam = PlayersStats;
    if len(Team) == 0:
        return px.scatter(), descriptionTime, descriptionHeight
    if "Wszyscy" not in Team:
        PickdTeam = pd.DataFrame()
        for t in Team:
            PickdTeam = PickdTeam.append(PlayersStats[PlayersStats["Obecny klub"] == t ])
    PickdTeam = PickdTeam[(PickdTeam["Wzrost"] >=Height[0]) &  (PickdTeam["Wzrost"] <= Height[1])]
    try:
        figStat = px.scatter(PickdTeam[(PickdTeam['Pozycja'].isnull() == False) & (PickdTeam[Stat].isnull() == False) ], x="nazwisko", y=Stat, color="Pozycja")
    except KeyError:
        figStat = px.scatter()
        
    return figStat, descriptionTime, descriptionHeight
@app3.callback(
    Output(component_id="StatsPlot2", component_property="figure"),
    Output(component_id="HeightLabel2", component_property="children"),
    Output(component_id="AgeLabel", component_property="children"),
    [
        Input("Team3",'value'),
        Input("Age",'value'),
        Input("Height2", "value"),
        
    ],
)
def CreateDashboard2(Team3, Age, Height2):
    #add description
    descriptionAge = "Wiek od " + str(Age[0]) + " do " + str(Age[1])
    descriptionHeight = "Wzrost od " + str(Height2[0]) + " do " + str(Height2[1])
    
    if len(Team3) == 0:
        return px.scatter(), descriptionHeight, descriptionAge
    
    PickTeam = PlayersAge
    if "Wszyscy" not in Team3:
        PickTeam = pd.DataFrame()
        for t in Team3:
            PickTeam = PickTeam.append(PlayersAge[PlayersAge["Obecny klub"] == t ])
            
    PickTeam = PickTeam[(PickTeam["Wzrost"] >=Height2[0]) &  (PickTeam["Wzrost"] <= Height2[1])]
    PickTeam = PickTeam[(PickTeam["Wiek"] >=Age[0]) &  (PickTeam["Wiek"] <= Age[1])] 
    try:
        figStats = px.scatter(PickTeam[(PickTeam['Noga'].isnull() == False) & (PickTeam['Bramka'].isnull() == False) ], x="nazwisko", y="Wiek", size="Bramka", color="Noga")
    except KeyError:
        figStats = px.scatter()
    
    return figStats, descriptionHeight, descriptionAge
@app3.callback(
    Output(component_id="Alert1", component_property="children"),
    Output(component_id="Alert2", component_property="children"),
    Output(component_id="Alert3", component_property="children"),
    Output(component_id="Alert4", component_property="children"),
    Output(component_id="Alert5", component_property="children"),
    Output(component_id="Alert6", component_property="children"),
    Output(component_id="TimeLabel2", component_property="children"),
    [
        Input("Time2",'value'),
        
    ],
)
def forAlerts(Time):
    #add description
    descriptionTime = "Minuty od " + str(Time[0]) + " do " + str(Time[1])
    
    limitedStats = Summary[(Summary["Czas"] >=Time[0]) &  (Summary["Czas"] <= Time[1])]
    Scored = limitedStats[(limitedStats["Co"] == "Gol") & (limitedStats["Info"] != "samobój")].groupby(by=["Kto"])["Co"].count().reset_index()
    ScoredPenalty = limitedStats[(limitedStats["Co"] == "Gol") & (limitedStats["Info"] == "rzut karny")].groupby(by=["Kto"])["Co"].count().reset_index()
    MissedPenalty = limitedStats[(limitedStats["Co"] == "Niestrzelony karny")].groupby(by=["Kto"])["Co"].count().reset_index()
    OwnGoal = limitedStats[(limitedStats["Co"] == "Gol") & (limitedStats["Info"] == "samobój")].groupby(by=["Kto"])["Co"].count().reset_index()
    Yellow = limitedStats[(limitedStats["Co"] == "Zolta kartka")].groupby(by=["Kto"])["Co"].count().reset_index()
    Red = limitedStats[(limitedStats["Co"] == "Czerwona")].groupby(by=["Kto"])["Co"].count().reset_index()

    #rename columns
    Scored.columns = ['id', "Bramka"] 
    ScoredPenalty.columns = ['id', "Karny"]
    MissedPenalty.columns = ["id", "Pudlo"]
    OwnGoal.columns = ['id','Samoboj']
    Yellow.columns = ['id', "Zolta"]
    Red.columns = ['id', "Czerwona"]

    #count overallstats
    goalsSum = Scored["Bramka"].sum()
    penaltySum = ScoredPenalty['Karny'].sum()
    yellowSum = Yellow["Zolta"].sum()
    redSum = Red['Czerwona'].sum()
    ownSum = OwnGoal['Samoboj'].sum()
    missedPenaltySum = MissedPenalty['Pudlo'].sum()
    summedStats = {"goalsSum": goalsSum, "penaltySum": penaltySum, "yellowSum": yellowSum,"redSum": redSum,"ownSum": ownSum,"missedPenaltySum": missedPenaltySum}
    
    a1 = "Bramki: "+ str(summedStats.get("goalsSum"))
    a2 = "Karne strzelone: "+ str(summedStats.get("penaltySum"))
    a3 = "Karne niestrzelone: "+ str(summedStats.get("missedPenaltySum"))          
    a4 = "Żółte kartki: "+ str(summedStats.get("yellowSum"))
    a5 = "Czerwone kartki: "+ str(summedStats.get("redSum"))
    a6 = "Samobóje: "+ str(summedStats.get("ownSum"))                           
    return a1,a2,a3,a4,a5,a6,descriptionTime
@app3.callback(
    Output(component_id="AgeLabel2", component_property="children"),
    Output(component_id="datatable-interactivity", component_property="data"),
    [
        Input("Team4",'value'),
        Input("Age2",'value'),
        
    ],
)
def CreateTable(Team, Age):
    descriptionAge = "Wiek od " + str(Age[0]) + " do " + str(Age[1])
    limitedAge = PlayersForTab[(PlayersForTab["Wiek"] >=Age[0]) &  (PlayersForTab["Wiek"] <= Age[1])]
    
    PickTeam = limitedAge
    if "Wszyscy" not in Team or len(Team) == 0:
        PickTeam = pd.DataFrame()
        for t in Team:
            PickTeam = PickTeam.append(limitedAge[limitedAge["Obecny klub"] == t ])
            
    return descriptionAge, PickTeam.to_dict("records")
app3.run_server(port=8053)

Dash app running on http://127.0.0.1:8053/
